In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from time import sleep
from random import randint
from datetime import datetime

In [16]:
def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

In [17]:
url = get_url("data science", "la")

In [18]:
response = requests.get(url)

In [19]:
response

<Response [200]>

In [32]:
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'job_seen_beacon')

In [33]:
len(cards)

15

In [34]:
card = cards[1]

In [48]:
atag = card.h2.a

In [49]:
atag

<a aria-label="full details of Junior Data Scientist" class="jcs-JobTitle" data-hide-spinner="true" data-hiring-event="false" data-jk="d9a532dde1fd0282" data-mobtk="1g2pncpf5t520800" href="/rc/clk?jk=d9a532dde1fd0282&amp;fccid=9e215d88a6b33622&amp;vjs=3" id="job_d9a532dde1fd0282" role="button" target="_blank"><span title="Junior Data Scientist">Junior Data Scientist</span></a>

In [50]:
atag.span

<span title="Junior Data Scientist">Junior Data Scientist</span>

In [51]:
job_title = atag.span.get("title")
job_title

'Junior Data Scientist'

In [54]:
job_url = 'https://www.indeed.com' + atag.get("href")

In [55]:
job_url

'https://www.indeed.com/rc/clk?jk=d9a532dde1fd0282&fccid=9e215d88a6b33622&vjs=3'

In [83]:
company = card.find("span", {"class": "companyName"}).text
company

'Deloitte'

In [72]:
job_location = card.find("div", {"class": "companyLocation"}).text
job_location

'New Orleans, LA 70139 (Central Business District area)'

In [86]:
job_summary = card.find("div", "job-snippet").text
job_summary

'Do you have a passion for artificial intelligence, machine learning, and data analysis? Do you yearn to have the impact of your work recognized and valued by…'

In [90]:
post_date = card.find("span", "date").text
post_date

'Posted12 days ago'

In [93]:
today = datetime.today().strftime("%Y-%m-%d")
today

'2022-05-11'

## Generalizing

In [103]:
def get_record(card):
    
    job_title = atag.span.get("title")
    company = card.find("span", {"class": "companyName"}).text
    job_location = card.find("div", {"class": "companyLocation"}).text
    post_date = card.find("span", "date").text
    today = datetime.today().strftime("%Y-%m-%d")
    job_summary = card.find("div", "job-snippet").text
    job_url = 'https://www.indeed.com' + atag.get("href")
    
    record = (job_title, company, job_location, post_date, today, job_summary, job_url)
    return record

In [104]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [107]:
records[1]

('Junior Data Scientist',
 'Deloitte',
 'New Orleans, LA 70139 (Central Business District area)',
 'Posted12 days ago',
 '2022-05-11',
 'Do you have a passion for artificial intelligence, machine learning, and data analysis? Do you yearn to have the impact of your work recognized and valued by…',
 'https://www.indeed.com/rc/clk?jk=d9a532dde1fd0282&fccid=9e215d88a6b33622&vjs=3')

## Gettting to the next page

In [111]:
page = soup.find("div", "pagination")
page

<div class="pagination" onmousedown="pclk(event);">
<ul class="pagination-list">
<li><b aria-current="true" aria-label="1" tabindex="0">1</b></li><li><a aria-label="2" data-pp="gQAPAAAAAAAAAAAAAAAB1CNazwAUAQAEbIoVcExysZSSuWO6jnv6RuQAAA" href="/jobs?q=data+science&amp;l=la&amp;start=10" onmousedown="addPPUrlParam &amp;&amp; addPPUrlParam(this);" rel="nofollow"><span class="pn">2</span></a></li><li><a aria-label="3" data-pp="gQAeAAAAAAAAAAAAAAAB1CNazwAaAQEBCQKBAB0G57wfSoQwR4UfVqf14qYjSwcAAA" href="/jobs?q=data+science&amp;l=la&amp;start=20" onmousedown="addPPUrlParam &amp;&amp; addPPUrlParam(this);" rel="nofollow"><span class="pn">3</span></a></li><li><a aria-label="4" data-pp="gQAtAAAAAAAAAAAAAAAB1CNazwAWAQEBBwDTSrC7u2oYIOKdvSV5s2KhiQAA" href="/jobs?q=data+science&amp;l=la&amp;start=30" onmousedown="addPPUrlParam &amp;&amp; addPPUrlParam(this);" rel="nofollow"><span class="pn">4</span></a></li><li><a aria-label="Next" data-pp="gQAPAAAAAAAAAAAAAAAB1CNazwAUAQAEbIoVcExysZSSuWO6jnv6RuQAAA" 

In [112]:
page.a.get("href")

'/jobs?q=data+science&l=la&start=10'

In [113]:
url = 'https://www.indeed.com' + page.a.get("href")
url

'https://www.indeed.com/jobs?q=data+science&l=la&start=10'

In [122]:
#while True:
for x in range(1):
    try:
        url = 'https://www.indeed.com' + page.a.get("href")
    except attributeError:
        break
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')
    
    for card in cards:
        record = get_record(card)
        records.append(record)
    

In [123]:
len(records)

1005

In [121]:
records[989]

('Junior Data Scientist',
 'Change Healthcare',
 'Remote in Louisiana+1 location',
 'Posted30+ days ago',
 '2022-05-11',
 '\nCollects data to identify root cause of problems.\nDevelops metrics that provide data for process measurement, identifying indicators for future improvement…\n',
 'https://www.indeed.com/rc/clk?jk=d9a532dde1fd0282&fccid=9e215d88a6b33622&vjs=3')

## Putting all together

In [144]:
from bs4 import BeautifulSoup
import requests
import csv
from time import sleep
from random import randint
from datetime import datetime

def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    
    job_title = atag.span.get("title")
    company = card.find("span", {"class": "companyName"}).text
    job_location = card.find("div", {"class": "companyLocation"}).text
    post_date = card.find("span", "date").text
    today = datetime.today().strftime("%Y-%m-%d")
    job_summary = card.find("div", "job-snippet").text
    job_url = 'https://www.indeed.com' + atag.get("href")
    
    record = (job_title, company, job_location, post_date, today, job_summary, job_url)
    return record

def main(position, location):
    records = []
    url = get_url(position, location)
    
    for x in range(1):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
    
        for card in cards:
            record = get_record(card)
            records.append(record)
    
        try:
            url = 'https://www.indeed.com' + page.a.get("href")
        except attributeError:
            break
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Posting Date', 'Extract Date', 'Summary', 'Job Url'])
        writer.writerows(records)
    

In [145]:
main("data engineer", "la")

In [146]:
import pandas as pd

df = pd.read_csv("results.csv")
df

,Job Title,Company,Location,Posting Date,Extract Date,Summary,Job Url
0,Junior Data Scientist,"Amazon Web Services, Inc.",Louisiana+2 locations,Posted30+ days ago,2022-05-12,\nHands on experience working on large-scale d...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
1,Junior Data Scientist,IBM,"Baton Rouge, LA 70801+3 locations",Posted13 days ago,2022-05-12,\nYou analyze and organize the data needed and...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
2,Junior Data Scientist,First Horizon Bank,"Lafayette, LA 70501",Posted30+ days ago,2022-05-12,"\nWork closely with other data engineers, busi...",https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
3,Junior Data Scientist,Tulane University,"New Orleans, LA",Posted21 days ago,2022-05-12,\nHighly proficient data analysis and data man...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
4,Junior Data Scientist,Pearson,"Baton Rouge, LA 70806",Posted30+ days ago,2022-05-12,"\nDesign, code, unit test, and deploy data pro...",https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
5,Junior Data Scientist,Blue Cross Blue Shield of Louisiana,"Baton Rouge, LA 70806+1 location",Posted9 days ago,2022-05-12,\nCompletion of a data engineer or relevant in...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
6,Junior Data Scientist,Noodle,"Remote in New Orleans, LA 70118+1 location",Posted30+ days ago,2022-05-12,\nBuilding Tableau visualizations to make data...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
7,Junior Data Scientist,Change Healthcare,Louisiana+6 locations,Posted30+ days ago,2022-05-12,\nWork with team members including data analys...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
8,Junior Data Scientist,Deloitte,"New Orleans, LA 70139 (Central Business Distri...",PostedJust posted,2022-05-12,"\nCollaborate with fellow engineers, engineeri...",https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...
9,Junior Data Scientist,Deloitte,"New Orleans, LA 70139 (Central Business Distri...",Posted30+ days ago,2022-05-12,\nImplement large-scale data ecosystems includ...,https://www.indeed.com/rc/clk?jk=d9a532dde1fd0...


In [142]:
df["Summary"][0]

'\nHands on experience working on large-scale data science/data analytics projects.\nHands on experience with building data or machine learning pipeline.\n'

In [143]:
df["Summary"][12]

'\nThe role gives opportunity to be part of an expert group of elite distributed systems developers/engineers, who have cutting edge expertise in distributed…\n'